# Faster than the clock algorithms

Faster than the clock algorithms are useful in cases when the acceptance rate of the standard Monte Carlo is low. An example of this is the Ising model at low temperatures, when we are disregarding most of the moves. The idea of the faster than the clock algorithms is to speed up the simulation by calculating the probability distribution of the number of moves that will be required in order to change the state of the system. Then, only accepted moves need to be evaluated during the simulation. One just has to properly generate the times between two of these accepted moves.

## Random depositions

Let's consider the problem of random depositions in one dimension. Consider the line of length $L$ on which we are randomly depositing  pins of width $2 \times \sigma$ on the interval $[\sigma,L-\sigma]$ which cannot overlap. At time $t=1$ we deposit the first pin and in succesive time steps we try to deposite further pins. We make the deposition if the move is allowed, otherwise we choose a new position etc. The process halts when no further pins can be deposited.

<br>
<table><tr style="background-color: white;">
<td> <img src="https://gist.github.com/mferrero/ae328ab0e3a0d3d7181a007daf5a373a/raw/clothes_pins_many.png" width=500 height=150 /> </td>
<td> <img src="https://gist.github.com/mferrero/ae328ab0e3a0d3d7181a007daf5a373a/raw/clothes_pins_single.png" width=200 height=150 /> </td>
</tr></table>
<br>

In the following, we will write two Monte Carlo algorithms to simulate random depositions, a direct real-time implementation and a faster than the clock implementation. Results will be compared with the analytical solution.

We first import the necessary libraries

In [1]:
import numpy as np
import bisect
import matplotlib.pylab as plt

## Real time algorithm

Write the random depositions algorithm in real time, which outputs the times at which the depositions are made.
You can use pins with $\sigma = 1$. Start with $L = 100$ and increase to $L = 10000$ to see how the
simulation time changes.

Hint: It is a good idea to encode the positions of the pins in an *ordered* list. You can then use bisection to quickly find where a new pin has to be inserted. This example shows how to use the `bisect` function
```python
pins = [1, 4, 6, 7, 10, 16]
ind = bisect.bisect(pins, 9) # find where 9 would go
pins.insert(ind, 9) # insert 9 in the right place
print(pins)
```

In [2]:
# parameters
sigma = 1
L = 10000
np.random.seed(9827)

# initialize variables
pins = [-sigma, L+sigma] # two fake pins
time = 0

# main loop
while(True):
    
    time += 1
    r = np.random.uniform(sigma, L-sigma)
    ind = bisect.bisect(pins, r)
    
    # distance to neighbors
    dist_r = pins[ind] - r
    dist_l = r - pins[ind-1]
    
    # there is enough space
    if dist_r >= 2*sigma and dist_l >= 2*sigma:

        # insert pin
        pins.insert(ind, r)

        # is there still some space somewhere?
        pin_array = np.array(pins)
        n = len(pins)
        intervals = pin_array[1:n] - pin_array[0:n-1]
        available = np.any(intervals > 4 * sigma)
        n_available = np.sum(intervals > 4 * sigma)
        
        print("time = ", time, "# pins = ", n-2, "# available = ", n_available)
        
        # stop if no space left
        if not available: break
        
            
print("Density = ", (len(pins)-2) * 2 * sigma / L)

time =  1 # pins =  1 # available =  2
time =  2 # pins =  2 # available =  3
time =  3 # pins =  3 # available =  4
time =  4 # pins =  4 # available =  5
time =  5 # pins =  5 # available =  6
time =  6 # pins =  6 # available =  7
time =  7 # pins =  7 # available =  8
time =  8 # pins =  8 # available =  9
time =  9 # pins =  9 # available =  10
time =  10 # pins =  10 # available =  11
time =  11 # pins =  11 # available =  12
time =  12 # pins =  12 # available =  13
time =  13 # pins =  13 # available =  14
time =  14 # pins =  14 # available =  15
time =  15 # pins =  15 # available =  16
time =  16 # pins =  16 # available =  17
time =  17 # pins =  17 # available =  18
time =  18 # pins =  18 # available =  19
time =  19 # pins =  19 # available =  20
time =  20 # pins =  20 # available =  21
time =  21 # pins =  21 # available =  22
time =  22 # pins =  22 # available =  23
time =  23 # pins =  23 # available =  24
time =  24 # pins =  24 # available =  25
time =  25 # pins 

## Faster than the clock algorithm

As you must have seen, the real-time implementation of the algorithm can become very slow. Indeed,
the probability to place the pins at the right places when there is not so much space left becomes
very small. As a result, the simulation times can become very long. We will now see how this can
be avoided by implementing a faster than the clock algorithm. Here are the steps:

- Given a certain configuration of pins, make a list of all the intervals where new pins can
  be inserted.
  
- From the cumulative length of all these intervals, you can easily deduce the probability that the
  next trial insertion will be accepted. You then also know the probability $p_r$ that this trial
  will be rejected.

- The distribution of the number of steps $t$ until a proposal is accepted is given by

  $$
  \pi(t) = p_r^{t-1} (1 - p_r)
  $$
  
  This distribution can be sampled from a uniform random variable $\xi \in [0,1]$ with
  
  $$
  t = 1 + \left\lfloor \frac{\log(\xi)}{\log(p_r)} \right\rfloor
  $$

  With the knowledge of $p_r$ you can generate a faster than the clock time until the next
  accepted proposal.
  
- Produce a new configuration of pins by randomly choosing a valid position for the new pin.
  Be careful to pick this position uniformly across all valid intervals that you have found above.
  
- Keep track of the times when pins were inserted.

Hint: It is a good idea to construct the list of intervals by specifying both their length and the position on the line where they start. This will be useful to more easily find the place where the next pin will be inserted.

In [ ]:
def compute_intervals(pins, sigma):
    """Given a configuration of pins, compute a list of all intervals where a further pin
       can be inserted. Returns both the length and the start of these intervals."""
    
    n_pins = len(pins)
    pin_array = np.array(pins)
    intervals = pin_array[1:n_pins] - pin_array[0:n_pins-1] - 4 * sigma
    starts = pin_array[0:n_pins-1] + 2 * sigma
    viable = intervals > 0
    
    return intervals[viable], starts[viable]

In [ ]:
# parameters
sigma = 1
L = 10000

# initialize variables
pins = [-sigma, L+sigma]
times = [0]
rho = [0]
time = 0

# there is still some free interval
while(True):
    
    # compute intervals - stop if no interval left
    interval_length, interval_start = compute_intervals(pins, sigma)
    if not np.any(interval_length): break

    # cumulative length of allowed positions
    tot_length = np.sum(interval_length)
    
    # rejection probability
    p_reject = 1 - tot_length / (L - 2*sigma)
    
    # find next allowed position
    r = np.random.uniform(0, tot_length)
    cum_length = np.cumsum(interval_length)
    ind = bisect.bisect(cum_length, r)
    delta = r - cum_length[ind-1] if ind > 0 else r
    position = interval_start[ind] + delta

    # insert pin
    bisect.insort(pins, position)
    
    # compute faster than the clock time
    time += 1
    if p_reject > 0:
        time += int(np.log(np.random.rand()) / np.log(p_reject))
        
    # store times and densities
    times.append(time)
    rho.append((len(pins)-2) * 2 * sigma / L)
        
print("Density = ", (len(pins)-2) * 2 * sigma / L)

## Comparison with analytical solution

It can be shown that the distribution of the density of pins as a function of
time is given by

$$
\rho(T) = \int_0^T  du\;\exp\left\{-2\int_{0}^u\frac{dv}{v}(1-e^{-v})\right\}
$$

where $T = 2\sigma t/L$ is a renormalized simulation time. Check if the faster than
the clock algorithm has produced densities in time that are compatible with this
formula. You can use this code to produce the analytical data:

```python
from scipy.integrate import quad

F = lambda u: quad(lambda v: (1 - np.exp(-v)) / v, 0, u)[0]
analytic = lambda T: quad(lambda u: np.exp(-2 * F(u)), 0, T)[0]
x_data = np.logspace(-4, 4, 50)
y_data = np.vectorize(analytic)(x_data)
```

In [ ]:
from scipy.integrate import quad

F = lambda u: quad(lambda v: (1 - np.exp(-v)) / v, 0, u)[0]
analytic = lambda T: quad(lambda u: np.exp(-2 * F(u)), 0, T)[0]
x_data = np.logspace(-4, 4, 50)
y_data = np.vectorize(analytic)(x_data)

plt.semilogx(np.array(times) * 2 * sigma / L, rho, '-r', lw=2)
plt.semilogx(x_data, y_data, '--', lw=2)

plt.ylabel(r'$\rho(t)$', size=20)
plt.xlabel(r'$2\sigma t/L$', size=20)
plt.title("Comparison with theory", fontsize=20);

## References

* Rényi, A. "On a One-Dimensional Problem Concerning Random Space-Filling." Publ. Math. Inst. Hung. Acad. Sci. 3, 109-127, 1958 
* Y. Pomeau, Some asymptotic estimates in the random parking problem, J. Phys. A: Math. Gen. 13 L193 (1980)
* V. Privman, J.-S. Wang, and P. Nielaba, Continuum limit in random sequential adsorption, Phys. Rev. B 43 3366 (1991)